# Redes Neuronales
### Tutorial de Keras 
####  Josué Irad Galindo De la Serna

In [1]:
# - - coding: utf- 8 - 
#Comenzamos importanto las librerías y funciones necesarias 
import pandas as pd
import numpy as np 

from sklearn.model_selection  import train_test_split
from sklearn.preprocessing    import StandardScaler
from sklearn.preprocessing    import MultiLabelBinarizer
from sklearn.metrics          import classification_report

from tensorflow.keras.models  import Sequential
from tensorflow.keras.layers  import Dense 


In [14]:
#Cargamos el dataset a un data frame de pandas 
file_path = '~/Documents/NeuralNetworks/auto.csv'
df = pd.read_csv(file_path)
df.head() #queremos ver las 5 primeras líneas

,compactness,circularity,dist_circularity,radius_ratio,pr_axis,max_length_ratio,scatter_ratio,elongatedness,pr_axis_rect,max_leng_regularity,scaled_var_major,scaled_var_minor,scaled_rad_gyration,skewness_major,skewness_minor,kurtosis_minor,kurtosis_major,hollows_ratio,class
0,95,48,83,178,72,10,162,42,20,159,176,379,184,70,6,16,187,197,van
1,91,41,84,141,57,9,149,45,19,143,170,330,158,72,9,14,189,199,van
2,104,50,106,209,66,10,207,32,23,158,223,635,220,73,14,9,188,196,saab
3,93,41,82,159,63,9,144,46,19,143,160,309,127,63,6,10,199,207,van
4,85,44,70,205,103,52,149,45,19,144,241,325,188,127,9,11,180,183,bus


In [132]:
df.tail()

,compactness,circularity,dist_circularity,radius_ratio,pr_axis,max_length_ratio,scatter_ratio,elongatedness,pr_axis_rect,max_leng_regularity,scaled_var_major,scaled_var_minor,scaled_rad_gyration,skewness_major,skewness_minor,kurtosis_minor,kurtosis_major,hollows_ratio,class
838,93,39,87,183,64,8,169,40,20,134,200,422,149,72,7,25,188,195,saab
839,89,46,84,163,66,11,159,43,20,159,173,368,176,72,1,20,186,197,van
840,106,54,101,222,67,12,222,30,25,173,228,721,200,70,3,4,187,201,saab
841,86,36,78,146,58,7,135,50,18,124,155,270,148,66,0,25,190,195,saab
842,85,36,66,123,55,5,120,56,17,128,140,212,131,73,1,18,186,190,van


Notamos que los datos son mediciones reales, entonces exceden los límites para utilizar las funciones de activación sigmoide o tangente hipebólica, por lo que será necesario excalarlos

In [15]:
#Eliminaremos la última columna (la de clasificación) para obtener los datos de entrada
datos = df.iloc[:,:-1]
#normalizamos
datos_s = StandardScaler().fit_transform(datos)
datos_s

array([[ 0.16236242,  0.51220379,  0.05795792, ...,  0.37859592,
        -0.3144815 ,  0.18254814],
       [-0.32429188, -0.62407988,  0.12141055, ...,  0.1547051 ,
         0.01000798,  0.45158325],
       [ 1.25733459,  0.83685627,  1.51736843, ..., -0.40502194,
        -0.15223676,  0.04803059],
       ...,
       [ 1.50066174,  1.48616123,  1.20010528, ..., -0.96474898,
        -0.3144815 ,  0.72061836],
       [-0.93260975, -1.43571108, -0.25930524, ...,  1.38610459,
         0.17225272, -0.08648697],
       [-1.05427333, -1.43571108, -1.02073681, ...,  0.60248673,
        -0.47672624, -0.75907474]])

In [16]:
#ahora vamos a identificar las clases usando la función get_dummies
dummies = pd.get_dummies(df)
clasif = dummies.iloc[:,-4:]
clasif

,class_bus,class_opel,class_saab,class_van
0,0,0,0,1
1,0,0,0,1
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
...,...,...,...,...
838,0,0,1,0
839,0,0,0,1
840,0,0,1,0
841,0,0,1,0


Tenemos ahora una variable de clasificación codificada como un vector que tiene un 1 en la i-ésima clase y un 0 en los demás lugares

Vamos ahora a dividir los datos como un conjunto de prueba y uno de aprendizaje.

In [17]:
Xtr,Xts,ytr,yts = train_test_split(datos_s,clasif)
print(Xtr.shape,ytr.shape)

print(Xts.shape,yts.shape)

(632, 18) (632, 4)
(211, 18) (211, 4)


## Comenzamos con Keras 
Ya que tenemos listos los datos, vamos a utilizar Keras para crear, entrenar y probar una red neuronal.

Consta de 2 modalidades para crear capas de redes: 

Sequential: funciona mediante el agregado de capas específicas que llevan acabo una función (como capa de neuronas Dense o Dropout)

Functional: Permite más versatilidad para crear capas 

### Crear la red 
Para crear una red necesitamos crear el modelo, agregar capas necesarias, definir la pérdida y el optimizador

In [105]:
'''
Definiremos una función para crear un modelo que recibe como argumentos
ni = # de capas de entrada 
ne = lista con el número de neuronas de las capas escondidas (ordenadas),
     donde el primer elemento es el más cercano a la capa de entrada
no = numero de capas de salida     

ni_fact = funcipon de activación en la capa de entrada 
            'sigmoid', 'relu', 'softmax'
ne_fact = vector de funcipon de activación en las capa ocultas
no_fact = funcipon de activación en la capa de entrada
'''

def create_model(ni=2,ne=[4],no=1,ni_fact = 'relu',ne_fact = 'relu',no_fact = 'softmax'):
    model = Sequential()
    model.add(Dense(ne[0], input_dim=ni, activation=ni_fact))
    
    if len(ne)>=2:
        for i in range(len(ne[1:])):
            #print(type(ne[i+1]),ne[i+1])
            model.add(Dense(ne[i+1], activation=ne_fact))
    else:
        print("solo hay 1 capa oculta")
        
    model.add(Dense(no,activation=no_fact))
    model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=['accuracy'])
    
    return model


In [117]:
ni = 18 #neuronas en la capa de entrada 
ne = [10,10] #neuronas en las capas ocultas
no = 4 #neuronas de la capa de salida 

nn1 = create_model(ni,ne,no)

<class 'int'> 10


In [118]:
nn1.summary()

Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_68 (Dense)             (None, 10)                190       
_________________________________________________________________
dense_69 (Dense)             (None, 10)                110       
_________________________________________________________________
dense_70 (Dense)             (None, 4)                 44        
Total params: 344
Trainable params: 344
Non-trainable params: 0
_________________________________________________________________


Procedemos a entrenar nuestro modelo 

In [129]:
#Entrenamos con 500 épocas 
nn1.fit(Xtr,ytr,epochs = 500, verbose = 1)

Epoch 1/500
20/20 [==============================] - 0s 735us/step - loss: 0.1459 - accuracy: 0.9430
Epoch 2/500
20/20 [==============================] - 0s 600us/step - loss: 0.1457 - accuracy: 0.9415
Epoch 3/500
20/20 [==============================] - 0s 679us/step - loss: 0.1450 - accuracy: 0.9383
Epoch 4/500
20/20 [==============================] - 0s 664us/step - loss: 0.1491 - accuracy: 0.9415
Epoch 5/500
20/20 [==============================] - 0s 647us/step - loss: 0.1497 - accuracy: 0.9415
Epoch 6/500
20/20 [==============================] - 0s 647us/step - loss: 0.1471 - accuracy: 0.9446
Epoch 7/500
20/20 [==============================] - 0s 737us/step - loss: 0.1447 - accuracy: 0.9399
Epoch 8/500
20/20 [==============================] - 0s 631us/step - loss: 0.1464 - accuracy: 0.9430
Epoch 9/500
20/20 [==============================] - 0s 638us/step - loss: 0.1455 - accuracy: 0.9446
Epoch 10/500
20/20 [==============================] - 0s 610us/step - loss: 0.1438 - accura

20/20 [==============================] - 0s 597us/step - loss: 0.1249 - accuracy: 0.9478
Epoch 161/500
20/20 [==============================] - 0s 615us/step - loss: 0.1251 - accuracy: 0.9541
Epoch 162/500
20/20 [==============================] - 0s 535us/step - loss: 0.1249 - accuracy: 0.9525
Epoch 163/500
20/20 [==============================] - 0s 570us/step - loss: 0.1226 - accuracy: 0.9509
Epoch 164/500
20/20 [==============================] - 0s 591us/step - loss: 0.1224 - accuracy: 0.9525
Epoch 165/500
20/20 [==============================] - 0s 599us/step - loss: 0.1251 - accuracy: 0.9541
Epoch 166/500
20/20 [==============================] - 0s 559us/step - loss: 0.1243 - accuracy: 0.9541
Epoch 167/500
20/20 [==============================] - 0s 531us/step - loss: 0.1238 - accuracy: 0.9541
Epoch 168/500
20/20 [==============================] - 0s 599us/step - loss: 0.1260 - accuracy: 0.9557
Epoch 169/500
20/20 [==============================] - 0s 568us/step - loss: 0.1235 - a

20/20 [==============================] - 0s 518us/step - loss: 0.1092 - accuracy: 0.9604
Epoch 319/500
20/20 [==============================] - 0s 602us/step - loss: 0.1082 - accuracy: 0.9573
Epoch 320/500
20/20 [==============================] - 0s 613us/step - loss: 0.1081 - accuracy: 0.9589
Epoch 321/500
20/20 [==============================] - 0s 605us/step - loss: 0.1076 - accuracy: 0.9604
Epoch 322/500
20/20 [==============================] - 0s 589us/step - loss: 0.1062 - accuracy: 0.9620
Epoch 323/500
20/20 [==============================] - 0s 558us/step - loss: 0.1089 - accuracy: 0.9604
Epoch 324/500
20/20 [==============================] - 0s 568us/step - loss: 0.1100 - accuracy: 0.9589
Epoch 325/500
20/20 [==============================] - 0s 585us/step - loss: 0.1094 - accuracy: 0.9589
Epoch 326/500
20/20 [==============================] - 0s 582us/step - loss: 0.1118 - accuracy: 0.9589
Epoch 327/500
20/20 [==============================] - 0s 550us/step - loss: 0.1060 - a

20/20 [==============================] - 0s 576us/step - loss: 0.0970 - accuracy: 0.9604
Epoch 477/500
20/20 [==============================] - 0s 565us/step - loss: 0.0955 - accuracy: 0.9636
Epoch 478/500
20/20 [==============================] - 0s 631us/step - loss: 0.0968 - accuracy: 0.9668
Epoch 479/500
20/20 [==============================] - 0s 592us/step - loss: 0.0959 - accuracy: 0.9652
Epoch 480/500
20/20 [==============================] - 0s 572us/step - loss: 0.0952 - accuracy: 0.9684
Epoch 481/500
20/20 [==============================] - 0s 543us/step - loss: 0.0949 - accuracy: 0.9652
Epoch 482/500
20/20 [==============================] - 0s 610us/step - loss: 0.0934 - accuracy: 0.9636
Epoch 483/500
20/20 [==============================] - 0s 587us/step - loss: 0.0967 - accuracy: 0.9668
Epoch 484/500
20/20 [==============================] - 0s 579us/step - loss: 0.0963 - accuracy: 0.9668
Epoch 485/500
20/20 [==============================] - 0s 597us/step - loss: 0.1025 - a

In [130]:
#evaluamos 
nn1.evaluate(Xts,yts)

7/7 [==============================] - 0s 823us/step - loss: 0.7613 - accuracy: 0.8294


[0.7612723112106323, 0.829383909702301]

Ya quedó entrenado el modelo, ahora veamos qué tan bién predice lo que estamos buscando 

In [131]:
predics = nn1.predict(Xts)
print(np.round(predics))

[[0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0.

Finalmente vamos a comparar diferentes modelos

In [139]:
nn0 = create_model(ni,[12],no)
nn1 = create_model(ni,[10,10],no)
nn2 = create_model(ni,[10,30,10],no)
nn3 = create_model(ni,[4,4],no)
nn4 = create_model(ni,[3,4,5,6],no)

n_epocas = 500

for lbl,model in [('12h',nn0),('10h10h',nn1),('10h30h10h',nn2),('4h4h',nn3),('3h4h5h6h',nn4)]:
    print("Modelo con ", lbl," escondidas")
    model.fit(Xtr,ytr,epochs=n_epocas,verbose=0)
    model.evaluate(Xts,yts)
    preds = model.predict(Xts)
    reporte = classification_report(yts,np.round(preds))
    
    

solo hay 1 capa oculta
<class 'int'> 10
<class 'int'> 30
<class 'int'> 10
<class 'int'> 4
<class 'int'> 4
<class 'int'> 5
<class 'int'> 6
Modelo con  12h  escondidas
7/7 [==============================] - 0s 545us/step - loss: 0.4171 - accuracy: 0.8009
Modelo con  10h10h  escondidas


/home/jigalindo0104/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


7/7 [==============================] - 0s 656us/step - loss: 0.5418 - accuracy: 0.8199
Modelo con  10h30h10h  escondidas
7/7 [==============================] - 0s 729us/step - loss: 1.0306 - accuracy: 0.7583
Modelo con  4h4h  escondidas
7/7 [==============================] - 0s 599us/step - loss: 0.5385 - accuracy: 0.7678
Modelo con  3h4h5h6h  escondidas


/home/jigalindo0104/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


7/7 [==============================] - 0s 581us/step - loss: 0.8109 - accuracy: 0.7346


/home/jigalindo0104/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
